In [1]:
import sys
sys.path.append('/Users/spindicate/Documents/programming/zooscraper/')

In [2]:
import django
django.setup()

In [3]:
import os
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

In [87]:
import shutil
import math
import numpy as np
import pandas as pd
import unidecode
from datetime import datetime as dt
from django.db import transaction
from django.db.models import Q

from zooscraper.globals import max_bulk_create
from casestudy.models import Country, Strindex, Mobility, Cause, Travel, Pollutant, Region, AppleMobility

In [5]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from zooscraper.globals import max_bulk_create, ChromeInstantiator, CHROMEDRIVER_PATH, CHROME_BIN

In [6]:
%load_ext autoreload
%autoreload 2

# Apple Mobility Data

In [150]:
import unidecode
from casestudy.baseframe import make_baseframe
from casestudy.see19 import get_baseframe

In [142]:
baseframe = make_baseframe(save=True, test=True)

In [151]:
baseframe = get_baseframe(test=True)

2020-05-15-17-34-02
https://raw.githubusercontent.com/ryanskene/see19/master/testset/see19-TEST-2020-05-15-17-34-02.csv


In [566]:
# Merge in travel popularity
df_travel = pd.DataFrame(Travel.objects.filter(region__in=regions).annotate(travel_year=F('year')).values('region_id', 'travel_year', 'visitors'))
df_base = pd.merge(df_base, df_travel, how='left', on=['region_id']).sort_values(by=['region_id', 'date'])

df_gdp = pd.DataFrame(GDP.objects.filter(region__in=regions).annotate(gdp_year=F('year')).values('region_id', 'gdp_year', 'gdp'))
df_base = pd.merge(df_base, df_gdp, how='left', on=['region_id']).sort_values(by=['region_name', 'date'])
df_base.country_id = df_base.country_id.astype('int64')

df_base.date = pd.to_datetime(df_base.date).dt.tz_localize(None)

,geo_type,region,transportation_type,alternative_name,2020-01-13,2020-01-14,2020-01-15,2020-01-16,2020-01-17,2020-01-18,...,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13
0,country/region,Albania,driving,,100.0,95.30,101.43,97.20,103.55,112.67,...,43.69,42.61,43.11,46.13,45.78,41.59,45.39,NaN,NaN,49.19
1,country/region,Albania,walking,,100.0,100.68,98.93,98.46,100.85,100.13,...,49.59,46.44,52.84,52.37,48.10,44.86,68.87,NaN,NaN,61.79
2,country/region,Argentina,driving,,100.0,97.07,102.45,111.21,118.45,124.01,...,32.01,33.63,35.13,35.56,40.25,33.82,19.82,NaN,NaN,38.87
3,country/region,Argentina,walking,,100.0,95.11,101.37,112.67,116.72,114.14,...,21.35,22.63,23.84,23.84,30.63,24.84,15.58,NaN,NaN,28.33
4,country/region,Australia,driving,,100.0,102.98,104.21,108.63,109.08,89.00,...,62.51,64.04,66.19,71.34,67.64,50.96,63.56,NaN,NaN,71.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1153,sub-region,Ile-de-France,driving,,100.0,102.06,103.61,110.01,107.47,113.34,...,25.35,26.28,28.59,29.41,25.81,25.73,30.66,NaN,NaN,52.21
1154,sub-region,Orebro County,driving,Örebro län,100.0,101.70,105.33,108.03,115.57,123.96,...,110.43,111.31,109.47,113.98,129.26,131.12,111.86,NaN,NaN,110.54
1155,sub-region,Ostergotland County,driving,,100.0,96.87,98.81,103.48,109.64,113.50,...,110.19,107.62,108.60,113.11,127.38,130.76,115.57,NaN,NaN,111.18
1156,sub-region,Usti nad Labem,driving,Ústecký kraj,100.0,101.57,107.63,112.87,120.32,126.19,...,104.54,104.41,117.95,126.84,140.64,137.18,120.35,NaN,NaN,109.38


# Oxford Stringency Index Data

In [27]:
oxcov_url = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'

In [28]:
df = pd.read_csv(oxcov_url, parse_dates=['Date'], infer_datetime_format=True)

In [32]:
df[df.CountryCode == 'TWN']

,CountryName,CountryCode,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,C3_Cancel public events,C3_Flag,C4_Restrictions on gatherings,...,H3_Contact tracing,H4_Emergency investment in healthcare,H5_Investment in vaccines,M1_Wildcard,ConfirmedCases,ConfirmedDeaths,StringencyIndex,StringencyIndexForDisplay,LegacyStringencyIndex,LegacyStringencyIndexForDisplay
149,Taiwan,TWN,2020-01-01,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,0.00,0.00,0.00,0.00
300,Taiwan,TWN,2020-01-02,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,7.14,7.14,9.52,9.52
451,Taiwan,TWN,2020-01-03,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,7.14,7.14,9.52,9.52
602,Taiwan,TWN,2020-01-04,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,7.14,7.14,9.52,9.52
753,Taiwan,TWN,2020-01-05,0.0,NaN,0.0,NaN,0.0,NaN,0.0,...,0.0,0.0,0.0,NaN,0.0,0.0,7.14,7.14,9.52,9.52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17665,Taiwan,TWN,2020-04-26,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,429.0,6.0,NaN,29.36,NaN,35.24
17816,Taiwan,TWN,2020-04-27,0.0,NaN,0.0,NaN,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,429.0,6.0,NaN,29.36,NaN,35.24
17967,Taiwan,TWN,2020-04-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,429.0,6.0,NaN,29.36,NaN,35.24
18118,Taiwan,TWN,2020-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,429.0,6.0,NaN,29.36,NaN,35.24


In [19]:
df.columns

Index(['CountryName', 'CountryCode', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'M1_Wildcard', 'ConfirmedCases', 'ConfirmedDeaths', 'StringencyIndex',
       'StringencyIndexForDisplay', 'LegacyStringencyIndex',
       'LegacyStringencyIndexForDisplay'],
      dtype='object')

In [58]:
df = df[['CountryName', 'CountryCode', 'Date', 'C1_School closing',
       'C2_Workplace closing', 'C3_Cancel public events',
       'C4_Restrictions on gatherings', 'C5_Close public transport',
        'C6_Stay at home requirements', 'C7_Restrictions on internal movement',
        'C8_International travel controls', 'E1_Income support',
        'E2_Debt/contract relief', 'E3_Fiscal measures',
        'E4_International support', 'H1_Public information campaigns',
        'H2_Testing policy', 'H3_Contact tracing',
        'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
        'StringencyIndex',
]]
# df = df.fillna(0)

In [67]:
[i.split('_')[0].lower() for i in df.columns if '_' in i]

['c1',
 'c2',
 'c3',
 'c4',
 'c5',
 'c6',
 'c7',
 'c8',
 'e1',
 'e2',
 'e3',
 'e4',
 'h1',
 'h2',
 'h3',
 'h4',
 'h5']

In [69]:
strindex_objs = []
no_country = []
with transaction.atomic():
    for i, row in df.iterrows():
        factor_kwargs = {i.split('_')[0].lower() : float(factor) for i, factor in row.iteritems() if '_' in i}
        
        # Set nans to None for saving database
        factor_kwargs = {k:None if math.isnan(v) else v for k, v in factor_kwargs.items()}
        print (float(row['StringencyIndex']))
        try:
            country = Country.objects.get(alpha3=row['CountryCode'])
            kwargs = {
                'country': country,
                'date': row['Date'],
                'strindex': float(row['StringencyIndex']),
                **factor_kwargs,
            }
            strindex_objs.append(Strindex(**kwargs))
        except Country.DoesNotExist:
            no_country.append(row['CountryName'])

0.0


TypeError: Strindex() got an unexpected keyword argument 'c1'

In [15]:
# if save:
max_bulk_create(strindex_objs)

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Strindex.date received a naive datetime (2020-03-21 00:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Strindex.date received a naive datetime (2020-03-22 00:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Strindex.date received a naive datetime (2020-03-23 00:00:00) while time zone support is active.
  RuntimeWarning)
/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1368: RuntimeWarning: DateTimeField Strindex.date received a naive datetime (2020-0

In [10]:
Strindex.objects.all().delete()

(17168, {'casestudy.Strindex': 17168})

# Global Mobility

In [67]:
Mobility.objects.all().count()

14651

In [ ]:
# def update_mobi(save=False):
DOWNLOADS_PATH = '/Users/spindicate/Downloads/'
MOBI_PATH = '/Users/spindicate/Documents/docs/covid19/mobility/'

with ChromeInstantiator(headless=False) as chrome:
    url = 'https://www.google.com/covid19/mobility/'
    chrome.get(url)

    href = 'https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv?cachebust=911a386b6c9c230f'
    element = WebDriverWait(chrome, 5).until(
        EC.element_to_be_clickable((By.XPATH, '//a[@href="{}"]'.format(href)))
    )
    download = chrome.find_element_by_xpath('//a[@href="{}"]'.format(href))
    download.click()

oldfilename = [file for file in os.listdir(MOBI_PATH) if 'Mobility' in file][0]
filename = [file for file in os.listdir(DOWNLOADS_PATH) if 'Mobility' in file][0]

os.remove(MOBI_PATH + oldfilename)
shutil.move(DOWNLOADS_PATH + filename, MOBI_PATH + filename)

In [33]:
mobi_file = '/Users/spindicate/Documents/docs/covid19/mobility/Global_Mobility_Report.csv'
df = pd.read_csv(mobi_file, parse_dates=['date'], infer_datetime_format=True)

/Users/spindicate/Documents/programming/envs/zooenv/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
df[df.country_region_code == 'HK']

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
32964,HK,Hong Kong,NaN,NaN,2020-02-15,-26.0,-4.0,-26.0,-34.0,-18.0,14.0
32965,HK,Hong Kong,NaN,NaN,2020-02-16,-28.0,-12.0,-35.0,-36.0,-19.0,13.0
32966,HK,Hong Kong,NaN,NaN,2020-02-17,-23.0,-3.0,-14.0,-32.0,-21.0,15.0
32967,HK,Hong Kong,NaN,NaN,2020-02-18,-24.0,-5.0,-11.0,-33.0,-21.0,16.0
32968,HK,Hong Kong,NaN,NaN,2020-02-19,-24.0,-8.0,-11.0,-31.0,-17.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...
33022,HK,Hong Kong,NaN,NaN,2020-04-13,-26.0,7.0,13.0,-45.0,-60.0,25.0
33023,HK,Hong Kong,NaN,NaN,2020-04-14,-25.0,-1.0,-10.0,-32.0,-20.0,16.0
33024,HK,Hong Kong,NaN,NaN,2020-04-15,-24.0,-4.0,-7.0,-30.0,-17.0,15.0
33025,HK,Hong Kong,NaN,NaN,2020-04-16,-24.0,-6.0,-7.0,-29.0,-12.0,12.0


In [74]:
mobi_objs = []
for alpha2, df_group in df.groupby('country_region_code'):
    if alpha2 == 'BR':
        regions = [unidecode.unidecode(''.join(reg.split('State of '))) for reg in df_group.sub_region_1.unique() if str(reg) != 'nan']
    else:
        regions = df_group.sub_region_1.unique()

    regions_objs = Region.objects.filter((Q(name__in=regions) | Q(name_alt__in=regions)) & Q(country_key__alpha2=alpha2))                       

    if regions_objs:
        df_group = df_group[~df_group.sub_region_1.isnull()]
        regions = df_group.sub_region_1.unique()

        for region, df_reg in df_group.groupby('sub_region_1'):
            df_reg = df_reg.copy(deep=True)
            df_reg = df_reg[df_reg.sub_region_2.isnull()]
            df_reg = df_reg.fillna(0)
            print (df_reg.head(2))
            if alpha2 == 'BR':
                region = unidecode.unidecode(''.join(region.split('State of ')))

            if region == 'Trentino-South Tyrol':
                region = 'P.A. Trento'
            try:
                filt = ((Q(name=region) | Q(name_alt=region) | Q(name_alt__contains=region[:6])) & Q(country_key__alpha2=alpha2))
                region_obj = Region.objects.get(filt)

                for i, row in df_reg.iterrows():
                    print (row['date'])
                    kwargs = {
                        'region': region_obj,
                        'date': row['date'],
                        'retail_n_rec': row['retail_and_recreation_percent_change_from_baseline'],
                        'groc_n_pharm': row['grocery_and_pharmacy_percent_change_from_baseline'],
                        'parks': row['parks_percent_change_from_baseline'],
                        'transit': row['transit_stations_percent_change_from_baseline'],
                        'workplaces': row['workplaces_percent_change_from_baseline'],
                        'residential': row['residential_percent_change_from_baseline']
                    }
                    mobi_objs.append(Mobility(**kwargs))
            except Region.MultipleObjectsReturned:
                try: 
                    filt = ((Q(name=region)) & Q(country_key__alpha2=alpha2))
                    region_obj = Region.objects.get(filt)
                except:
                    filt = ((Q(name=region) | Q(name_alt=region)) & Q(country_key__alpha2=alpha2))
                    region_obj = Region.objects.get(filt)

                for i, row in df_reg.iterrows():
                    kwargs = {
                        'region': region_obj,
                        'date': row['date'],
                        'retail_n_rec': row['retail_and_recreation_percent_change_from_baseline'],
                        'groc_n_pharm': row['grocery_and_pharmacy_percent_change_from_baseline'],
                        'parks': row['parks_percent_change_from_baseline'],
                        'transit': row['transit_stations_percent_change_from_baseline'],
                        'workplaces': row['workplaces_percent_change_from_baseline'],
                        'residential': row['residential_percent_change_from_baseline']
                    }
                    mobi_objs.append(Mobility(**kwargs))
            except Region.DoesNotExist:
                print (region)
    else:
        df_group = df_group[df_group.sub_region_1.isnull()]
        df_group = df_group.fillna(0)
        country_name = df_group['country_region'].iloc[0]

        if country_name == 'South Korea':
            country_name = 'Korea, South'
        elif country_name == 'Puerto Rico':
            alpha2 = 'US'
        elif country_name == "Côte d'Ivoire":
            country_name = "Cote d'Ivoire"
            alpha2 = ''

        try:
            region_obj = Region.objects.get((Q(name=country_name) | Q(name_alt=country_name)) & Q(country_key__alpha2=alpha2))
            for i, row in df_group.iterrows():
                kwargs = {
                    'region': region_obj,
                    'date': row['date'],
                    'retail_n_rec': row['retail_and_recreation_percent_change_from_baseline'],
                    'groc_n_pharm': row['grocery_and_pharmacy_percent_change_from_baseline'],
                    'parks': row['parks_percent_change_from_baseline'],
                    'transit': row['transit_stations_percent_change_from_baseline'],
                    'workplaces': row['workplaces_percent_change_from_baseline'],
                    'residential': row['residential_percent_change_from_baseline']
                }
                mobi_objs.append(Mobility(**kwargs))
        except Region.DoesNotExist:
            print (country_name, alpha2)
            pass

if save:
    max_bulk_create(mobi_objs)

     country_region_code country_region                  sub_region_1  \
2961                  AU      Australia  Australian Capital Territory   
2962                  AU      Australia  Australian Capital Territory   

      sub_region_2       date  \
2961             0 2020-02-15   
2962             0 2020-02-16   

      retail_and_recreation_percent_change_from_baseline  \
2961                                                7.0    
2962                                                8.0    

      grocery_and_pharmacy_percent_change_from_baseline  \
2961                                                5.0   
2962                                               19.0   

      parks_percent_change_from_baseline  \
2961                                28.0   
2962                                34.0   

      transit_stations_percent_change_from_baseline  \
2961                                            4.0   
2962                                           16.0   

      workplaces_perce

2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00
2020-03-10 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00
2020-03-18 00:00:00
2020-03-19 00:00:00
2020-03-20 00:00:00
2020-03-21 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-24 00:00:00
2020-03-25 00:00:00
2020-03-26 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00
2020-04-01 00:00:00
2020-04-02 00:00:00
2020-04-03 00:00:00
2020-04-04 00:00:00


     country_region_code country_region      sub_region_1  sub_region_2  \
6553                  BR         Brazil  Federal District             0   
6554                  BR         Brazil  Federal District             0   

           date  retail_and_recreation_percent_change_from_baseline  \
6553 2020-02-15                                               10.0    
6554 2020-02-16                                               11.0    

      grocery_and_pharmacy_percent_change_from_baseline  \
6553                                                7.0   
6554                                                6.0   

      parks_percent_change_from_baseline  \
6553                                -8.0   
6554                               -10.0   

      transit_stations_percent_change_from_baseline  \
6553                                            8.0   
6554                                            4.0   

      workplaces_percent_change_from_baseline  \
6553                              

     country_region_code country_region       sub_region_1  sub_region_2  \
6805                  BR         Brazil  State of Amazonas             0   
6806                  BR         Brazil  State of Amazonas             0   

           date  retail_and_recreation_percent_change_from_baseline  \
6805 2020-02-15                                                9.0    
6806 2020-02-16                                                4.0    

      grocery_and_pharmacy_percent_change_from_baseline  \
6805                                                7.0   
6806                                                5.0   

      parks_percent_change_from_baseline  \
6805                                -2.0   
6806                               -10.0   

      transit_stations_percent_change_from_baseline  \
6805                                           10.0   
6806                                           11.0   

      workplaces_percent_change_from_baseline  \
6805                           

2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00
2020-03-10 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00
2020-03-18 00:00:00
2020-03-19 00:00:00
2020-03-20 00:00:00
2020-03-21 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-24 00:00:00
2020-03-25 00:00:00
2020-03-26 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00
2020-04-01 00:00:00
2020-04-02 00:00:00
2020-04-03 00:00:00
2020-04-04 00:00:00


...

2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00
2020-03-18 00:00:00
2020-03-19 00:00:00
2020-03-20 00:00:00
2020-03-21 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-24 00:00:00
2020-03-25 00:00:00
2020-03-26 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00
2020-04-01 00:00:00
2020-04-02 00:00:00
2020-04-03 00:00:00
2020-04-04 00:00:00
2020-04-05 00:00:00
2020-04-06 00:00:00
2020-04-07 00:00:00
2020-04-08 00:00:00
2020-04-09 00:00:00
2020-04-10 00:00:00
2020-04-11 00:00:00
2020-04-12 00:00:00
2020-04-13 00:00:00
2020-04-14 00:00:00
2020-04-15 00:00:00
2020-04-16 00:00:00
2020-04-17 00:00:00
      country_region_code country_region sub_region_1  sub_region_2  \
40104                  IT          Italy     Calabria             0   
40105                  IT          Italy     Calabria             0   

            date  retail_a

2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00
2020-03-10 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00
2020-03-18 00:00:00
2020-03-19 00:00:00
2020-03-20 00:00:00
2020-03-21 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-24 00:00:00
2020-03-25 00:00:00
2020-03-26 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00
2020-04-01 00:00:00
2020-04-02 00:00:00
2020-04-03 00:00:00
2020-04-04 00:00:00


2020-02-15 00:00:00
2020-02-16 00:00:00
2020-02-17 00:00:00
2020-02-18 00:00:00
2020-02-19 00:00:00
2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00
2020-03-10 00:00:00
2020-03-11 00:00:00
2020-03-12 00:00:00
2020-03-13 00:00:00
2020-03-14 00:00:00
2020-03-15 00:00:00
2020-03-16 00:00:00
2020-03-17 00:00:00
2020-03-18 00:00:00
2020-03-19 00:00:00
2020-03-20 00:00:00
2020-03-21 00:00:00
2020-03-22 00:00:00
2020-03-23 00:00:00
2020-03-24 00:00:00
2020-03-25 00:00:00
2020-03-26 00:00:00
2020-03-27 00:00:00
2020-03-28 00:00:00
2020-03-29 00:00:00
2020-03-30 00:00:00
2020-03-31 00:00:00
2020-04-01 00:00:00
2020-04-02 00:00:00
2020-04-03 00:00:00
2020-04-04 00:00:00


2020-02-20 00:00:00
2020-02-21 00:00:00
2020-02-22 00:00:00
2020-02-23 00:00:00
2020-02-24 00:00:00
2020-02-25 00:00:00
2020-02-26 00:00:00
2020-02-27 00:00:00
2020-02-28 00:00:00
2020-02-29 00:00:00
2020-03-01 00:00:00
2020-03-02 00:00:00
2020-03-03 00:00:00
2020-03-04 00:00:00
2020-03-05 00:00:00
2020-03-06 00:00:00
2020-03-07 00:00:00
2020-03-08 00:00:00
2020-03-09 00:00:00

KeyboardInterrupt: 

# Causes of Death

In [12]:
"""ITALY"""
ital_causes = {'infectious': 1, 'neoplasms': 2, 'blood': 3, 'endo': 4, 'mental': 5, 'nervous': 6, 'circul': 7, 
'respir': 8, 'digest': 9, 'skin': 10, 'musculo': 11, 'genito': 12, 'childbirth': 13, 
'perinatal': 14, 'congenital': 15, 'other': 16, 'external': 17}
cause_map = {v: k for k, v in ital_causes.items()}

In [83]:
"""US CLASSIFICATION"""
us_causes = {
    'A': 1, 'B': 1, 'C': 2, '<D48': 2, '>D49': 3, 'E': 4, 'F': 5, 'G': 6, '<H57': 6, '>H60': 6, 
    'I': 7, 'J': 8, 'K': 9, 'L': 10, 'M': 11, 'N': 12, 'O': 13, 
    'P': 14, 'Q': 15, 'R': 16, 'U': 16, 'V': 17, 'W': 17, 'X': 17, 'Y': 17
}

In [201]:
"""GLOBAL"""
global_causes = {
    'Lower respiratory infections (deaths)': 1, 'Intestinal infectious diseases (deaths)': 1,
    'Protein-energy malnutrition (deaths)': 4, 'Terrorism (deaths)': 17,
    'Cardiovascular diseases (deaths)': 7, 'Dementia (deaths)': 5, 'Kidney disease (deaths)': 9, 
    'Respiratory diseases (deaths)': 8, 'Liver diseases (deaths)': 9, 'Digestive diseases (deaths)': 9,
    'Diabetes (deaths)': 4, 'Hepatitis (deaths)': 1, 'Cancers (deaths)': 2,
    'Parkinson disease (deaths)': 6, 'Fire (deaths)': 17, 'Malaria (deaths)': 1, 'Drowning (deaths)': 17,
    'Homicide (deaths)': 17, 'HIV/AIDS (deaths)': 1,  'Drug use disorders (deaths)': 17,
    'Road injuries (deaths)': 17, 'Tuberculosis (deaths)': 1, 'Maternal disorders (deaths)': 13,
    'Neonatal disorders (deaths)': 14, 'Alcohol use disorders (deaths)': 5,  'Natural disasters (deaths)': 17,
    'Diarrheal diseases (deaths)': 9, 'Heat (hot and cold exposure) (deaths)': 16,
    'Nutritional deficiencies (deaths)': 4, 'Suicide (deaths)': 17, 'Conflict (deaths)': 17, 'Poisonings (deaths)': 17
}

**Global**

In [202]:
global_file = '/Users/spindicate/Documents/docs/covid19/health/annual-number-of-deaths-by-cause.csv'
df = pd.read_csv(global_file)
excluded = ['USA', 'ITA', 'HKG']
countries_w_subregions = ['BRA', 'CAN', 'AUS', 'CHN']
df = df[(df.Year == 2017) & ~(df.Code.isin(excluded)) & (df.Code.notnull())]
df = df.fillna(0)

In [203]:
# Hong Kong to Chart by copying China stats
df = df[df.Entity != 'Hong Kong']
df = df.reset_index().drop(columns='index')

In [204]:
from django.db.models import Sum

In [205]:
cause_objs = []
for i, row in df.iterrows():
    try:
        country = Country.objects.get(alpha3=row.Code)

        if country.alpha3 in countries_w_subregions:
            print (country.alpha3)
            regions = Region.objects.filter(country_key__alpha3=country.alpha3)
            country_pop = regions.aggregate(Sum('population'))['population__sum']
            for region in regions:
                kwargs = {'year': row.Year, 'region': region}

                for j, cell in row.iteritems():
                    if j in global_causes.keys():
                        kwargs[cause_map[global_causes[j]]] = cell * region.population / country_pop
                cause_objs.append(Cause(**kwargs))
        else:
            kwargs = {'year': row.Year, 'country': country}

            for j, cell in row.iteritems():
                if j in global_causes.keys():
                    kwargs[cause_map[global_causes[j]]] = cell
            cause_objs.append(Cause(**kwargs))
    except:
        print (row.Code, row.Entity)

AUS
BRA
CAN
CHN
OWID_KOS Kosovo
OWID_WRL World


In [206]:
len(cause_objs)

269

In [207]:
Cause.objects.all().delete()

(350, {'casestudy.Cause': 350})

In [208]:
max_bulk_create(cause_objs)

**HONG KONG**

In [29]:
Cause.objects.filter(country__name='Hong Kong').values()

<QuerySet []>

In [22]:
hk = Country.objects.get(name='Hong Kong')
hk_file = '/Users/spindicate/Documents/docs/covid19/health/hong_kong_causes.csv'
df = pd.read_csv(hk_file, thousands=',')

In [23]:
df

,Cause,code,Deaths,Unnamed: 3
0,Malignant neoplasms,2.0,14354.0,NaN
1,Pneumonia,8.0,10079.0,NaN
2,Diseases of heart,7.0,6138.0,NaN
3,Cerebrovascular diseases,7.0,3124.0,NaN
4,Chronic lower respiratory diseases,8.0,2047.0,NaN
5,External causes of morbidity and mortality,17.0,1920.0,NaN
6,"Nephritis, nephrotic syndrome and nephrosis",6.0,1436.0,NaN
7,Dementia,5.0,1004.0,NaN
8,Septicaemia,1.0,880.0,NaN
9,Aortic aneurysm and dissection,3.0,425.0,NaN


In [28]:
cause_objs = []
cause_kwargs = {}
# df.Deaths = df.Deaths.astype('float64')
for code, df_group in df.groupby('code'):
    deaths = df_group.Deaths.sum()
    cause_kwargs[cause_map[code]] = deaths
    print (code, deaths)
    print (hk.id)
cause_objs.append(Cause(**{'year': 2017, 'country': hk, **cause_kwargs}))

1.0 880.0
100
2.0 14354.0
100
3.0 425.0
100
5.0 1004.0
100
6.0 1436.0
100
7.0 9262.0
100
8.0 12126.0
100
16.0 6519.0
100
17.0 1920.0
100


In [25]:
Country.objects.filter(id=100).value

AttributeError: 'QuerySet' object has no attribute 'value'

In [26]:
Cause.objects.filter(country_id=100).delete()

(1, {'casestudy.Cause': 1})

In [30]:
len(cause_objs)

1

In [31]:
max_bulk_create(cause_objs)

**US**

In [213]:
us_file = '/Users/spindicate/Documents/docs/covid19/health/us_causes_of_death_by_state_2018.txt'
cols = ['Notes', 'State', 'State Code', 'Cause of death', 'Cause of death Code', 'Deaths', 'Population', 'Crude', 'Rate']
df = pd.read_csv(us_file, sep='\t', names=cols, encoding= 'unicode_escape')
df = df.iloc[1:]
df.Deaths = df.Deaths.fillna(0).astype('int64')
df = df[df.State.notnull()].copy(deep=True)

In [214]:
def strip_code(deathcode):
    spread_codes = ['D', 'H']
    if deathcode[0] not in spread_codes:
            return deathcode[0]
    elif deathcode[0] == 'D':
        if int(deathcode[1:3]) <= 48:
            return '<D48' 
        else:
            return '>D49'
        kwargs[cause_map[us_causes[key]]] = row.Deaths
    elif deathcode[0] == 'H':
        if int(deathcode[1:3]) <= 48: 
            return '<H57'
        else:
            return '>H60'

In [215]:
df['Cause of death Code'] = np.vectorize(strip_code)(df['Cause of death Code'])

In [216]:
cause_objs = []
spread_codes = ['D', 'H']
for state, df_group in df.groupby('State'):
    region = Region.objects.get(name=state, country_key__alpha3='USA')
    kwargs = {'region': region, 'year': 2018}
    
    for code, df_code in df_group.groupby('Cause of death Code'):
        deaths = df_code.Deaths.sum()
        kwargs[cause_map[us_causes[code]]] = deaths
    cause_objs.append(Cause(**kwargs))

In [217]:
len(cause_objs)

51

In [219]:
max_bulk_create(cause_objs)

**ITA**

In [220]:
ital_file = '/Users/spindicate/Documents/docs/covid19/health/italy_cause_of_death.csv'
df = pd.read_csv(ital_file)
df = df.replace('Bolzano / Bozen', 'Bolzano/Bozen')
df = df.replace("Valle d'Aosta / Vallée d'Aoste", "Valle d'Aosta")
regions = [name for pair in Region.objects.filter(country_key__alpha3='ITA').values_list('name', 'name_alt') for name in pair if name]

df = df[(df.Territory.isin(regions)) & (df.TIPO_DATO15 == 'DEATH') & (df.Gender == 'total')]
df.CAUSEMORTE_SL = [int(cell[0]) for i, cell in df.CAUSEMORTE_SL.str.split('_').iteritems()]
df = df[df.CAUSEMORTE_SL.isin(cause_map.keys())]

In [221]:
df

,ITTER107,Territory,TIPO_DATO15,Data type,SEXISTAT1,Gender,CAUSEMORTE_SL,Underlying cause of death - European Short List,TIME,Select time,Value,Flag Codes,Flags
2134,ITC1,Piemonte,DEATH,deaths,9,total,2,of which malignant neoplasm of breast,2017,2017,1121.0,NaN,NaN
2135,ITC1,Piemonte,DEATH,deaths,9,total,2,of which malignant neoplasm of ovary,2017,2017,255.0,NaN,NaN
2136,ITC1,Piemonte,DEATH,deaths,9,total,7,of which acute myocardial infarction,2017,2017,1701.0,NaN,NaN
2137,ITC1,Piemonte,DEATH,deaths,9,total,7,cerebrovascular diseases,2017,2017,5805.0,NaN,NaN
2138,ITC1,Piemonte,DEATH,deaths,9,total,8,influenza,2017,2017,55.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70382,ITF5,Basilicata,DEATH,deaths,9,total,5,"drug dependence, toxicomania",2017,2017,1.0,NaN,NaN
70391,ITF2,Molise,DEATH,deaths,9,total,13,"complications of pregnancy, childbirth and pue...",2017,2017,1.0,NaN,NaN
70426,ITD10,Bolzano/Bozen,DEATH,deaths,9,total,5,"drug dependence, toxicomania",2017,2017,1.0,NaN,NaN
70869,ITF1,Abruzzo,DEATH,deaths,9,total,13,"complications of pregnancy, childbirth and pue...",2017,2017,1.0,NaN,NaN


In [222]:
cause_objs = []
for i, df_terr in df.groupby('Territory'):
    region = Region.objects.get(Q(name=df_terr.Territory.iloc[0]) | Q(name_alt=df_terr.Territory.iloc[0]))
    kwargs = {'region': region, 'year': df_terr.TIME.iloc[0]}
    
    for i, df_group in df_terr.groupby('CAUSEMORTE_SL'):
        kwargs[cause_map[int(i)]] = df_group.Value.sum()
    
    cause_objs.append(Cause(**kwargs))

In [223]:
len(cause_objs)

21

In [224]:
max_bulk_create(cause_objs)

# Travel

In [225]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'visitors_by_country.xlsx'

df = pd.read_excel(travel_path + travelfile, error_bad_lines=False)
df = df.replace('Hong Kong SAR, China', 'Hong Kong')
df = df.replace('Macao SAR, China', 'Macao')
df = df.replace('Korea', 'Korea, South')
df = df.replace('Myanmar', 'Burma')
df = df.replace('Dem. Rep. Congo', 'Democratic Republic of the Congo')
df = df.replace('Congo', 'Congo (Brazzaville)')

In [229]:
df[df.Country == 'Hong Kong']

,ank,Country,Value,Year
12,13,Hong Kong,27884000,2017


In [190]:
# sub_regions = ['Australia', 'Brazil', 'Canada', 'Italy', 'United States', 'China']

In [191]:
# df = df[~df['Country'].isin(sub_regions)]

In [232]:
Travel.objects.filter(region__name='Martinique')

<QuerySet []>

In [149]:
travel_objs = []
for i, row in df.iterrows():
    try:
        if row.Country == 'Georgia':
            region = Region.objects.get(name='Georgia', country='Georgia')
        else:
            region = Region.objects.get(Q(name=row.Country) | Q(name_alt=row.Country))

        kwargs = {
            'region': region,
            'year': row.Year,
            'visitors': row.Value,
        }
        travel_objs.append(Travel(**kwargs))
    except:
        print (row.Country)

Lesotho
Tajikistan
St. Lucia
The Gambia
Samoa
Palau
St. Kitts and Nevis
Vanuatu
St. Vincent and the Grenadines
Tonga
São Tomé and Principe
Comoros
Solomon Islands
Turkmenistan
Kiribati
Tuvalu


In [153]:
# max_bulk_create(travel_objs)

In [192]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'australia.xlsx'

df_aus = pd.read_excel(travel_path + travelfile, header=None)
df_aus.columns = ['region', 'visitors']

In [193]:
df_aus['%'] = df_aus.visitors / df_aus.visitors.sum()

In [194]:
df_aus

,region,visitors,%
0,Northern Territory,297,0.024827
1,Australian Capital Territory,248,0.020731
2,Tasmania,457,0.038201
3,Western Australia,961,0.080331
4,Queensland,2700,0.225696
5,Victoria,3000,0.250773
6,New South Wales,4300,0.359442


In [201]:
for i, row in df_aus.iterrows():
    ausvis = df[df.Country == 'Australia'].Value
    region = Region.objects.get(name=row.region)
    travel = Travel(region=region, year=2017, visitors=row['%']*ausvis)
    travel.save()

In [21]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'brazil.xlsx'

df_braz = pd.read_excel(travel_path + travelfile, header=None)
df_braz.columns = ['region', 'visitors']
df_braz['%'] = df_braz.visitors / df_braz.visitors.sum()
df_braz = df_braz.replace('Rio Grande Do Sol', 'Rio Grande Do Sul')

In [22]:
df_braz

,region,visitors,%
0,Sao Paulo,2250.00,0.340887
1,Rio De Janeiro,1293.00,0.195897
2,Rio Grande Do Sul,1087.00,0.164686
3,Parana,948.00,0.143627
4,Santa Catarina,226.00,0.034240
5,Bahia,148.00,0.022423
6,Pernambuco,121.00,0.018332
7,Ceara,107.00,0.016211
8,Mato Grosso Do Sul,91.00,0.013787
9,Minas Gerais,81.85,0.012401


In [23]:
for i, row in df_braz.iterrows():
    print (row.region)
    brazvis = df[df.Country == 'Brazil'].Value
    print (brazvis)
    region = Region.objects.get(Q(name=row.region) | Q(name_alt=row.region))
    travel = Travel(region=region, year=2017, visitors=row['%']*brazvis)
#     travel.save()

Sao Paulo
48    6589000
Name: Value, dtype: int64
Rio De Janeiro
48    6589000
Name: Value, dtype: int64
Rio Grande Do Sul
48    6589000
Name: Value, dtype: int64
Parana
48    6589000
Name: Value, dtype: int64
Santa Catarina
48    6589000
Name: Value, dtype: int64
Bahia
48    6589000
Name: Value, dtype: int64
Pernambuco
48    6589000
Name: Value, dtype: int64
Ceara
48    6589000
Name: Value, dtype: int64
Mato Grosso Do Sul
48    6589000
Name: Value, dtype: int64
Minas Gerais
48    6589000
Name: Value, dtype: int64
Distrito Federal
48    6589000
Name: Value, dtype: int64
Amazonas
48    6589000
Name: Value, dtype: int64
Amapa
48    6589000
Name: Value, dtype: int64
Acre
48    6589000
Name: Value, dtype: int64
Parana
48    6589000
Name: Value, dtype: int64
Rio Grande do Norte
48    6589000
Name: Value, dtype: int64
Roraima
48    6589000
Name: Value, dtype: int64


In [292]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'canada.csv'

df_cad = pd.read_csv(travel_path + travelfile)

In [294]:
df_cad['Geography, province visited'] = df_cad['Geography, province visited'].fillna(method='pad')

In [300]:
series = []
for i, cell in df_cad['2017'].iteritems():
    series.append(float(''.join(cell[:-1].split(','))))
df_cad['2017'] = series

In [311]:
agg_dicts = []
for i, df_group in df_cad.groupby('Geography, province visited'):
    total = df_group['2017'].sum()
    agg_dicts.append((i, total))

In [316]:
df_cad = pd.DataFrame(agg_dicts)
df_cad.columns = ['region', 'visitors']
df_cad['%'] = df_cad.visitors / df_cad.visitors.sum()

In [318]:
for i, row in df_cad.iterrows():
    print (row.region)
    cadvis = df[df.Country == 'Canada'].Value
    region = Region.objects.get(Q(name=row.region) | Q(name_alt=row.region))
    travel = Travel(region=region, year=2017, visitors=row['%']*cadvis)
    travel.save()

Alberta
British Columbia
Manitoba
New Brunswick
Newfoundland and Labrador
Nova Scotia
Ontario
Prince Edward Island
Quebec
Saskatchewan


In [322]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'italy.xlsx'

df_ita = pd.read_excel(travel_path + travelfile)
df_ita = df_ita.iloc[1:, 1:]

In [324]:
df_ita.columns = ['city', 'total', 'visitors', 'region']

In [332]:
agg_dicts = []
for i, df_group in df_ita.groupby('region'):
    total = df_group.visitors.sum()
    agg_dicts.append((df_group.region.iloc[0][1:], total))

In [347]:
df_ita = pd.DataFrame(agg_dicts)
df_ita.columns = ['region', 'visitors']
df_ita['%'] = df_ita.visitors / df_ita.visitors.sum()
df_ita = df_ita.replace('Aosta Valley', "Valle d'Aosta")
df_ita = df_ita.replace('Trentino-Alto Adige/Südtirol', "P.A. Trento")

In [349]:
for i, row in df_ita.iterrows():
    print (row.region)
    cadvis = df[df.Country == 'Italy'].Value
    region = Region.objects.get(Q(name=row.region) | Q(name_alt=row.region))
    travel = Travel(region=region, year=2017, visitors=row['%']*cadvis)
    travel.save()

Abruzzo
Valle d'Aosta
Apulia
Calabria
Campania
Emilia-Romagna
Friuli-Venezia Giulia
Lazio
Liguria
Lombardy
Marche
Piedmont
Sardinia
Sicily
P.A. Trento
Tuscany
Umbria
Veneto


In [379]:
travel_path = '/Users/spindicate/Documents/docs/covid19/travel/'
travelfile = 'usa.xlsx'

df_usa = pd.read_excel(travel_path + travelfile)
df_usa = df_usa.iloc[2:, [1,2]]
df_usa.columns = ['region', '%']

In [380]:
df_usa

,region,%
2,New York,0.2709
3,Florida,0.2351
4,California,0.2139
5,Nevada,0.0813
6,Hawaiian Islands,0.0798
7,Texas,0.0486
8,Massachusetts,0.046
9,Illinois,0.0406
10,Guam,0.0405
11,Arizona,0.0293


In [391]:
for i, row in df_usa.iterrows():
    region = row.region.strip()
    print (region)
    cadvis = df[df.Country == 'United States'].Value
    region = Region.objects.get((Q(name=region) | Q(name_alt=region)) & Q(country_key__alpha3='USA'))
    travel = Travel(region=region, year=2017, visitors=row['%']*cadvis)
    travel.save()

New York
Florida
California
Nevada
Hawaiian Islands
Texas
Massachusetts
Illinois
Guam
Arizona
New Jersey
Pennsylvania
Georgia
Washington
Utah
Colorado
North Carolina
Louisiana
Michigan
Virginia
Ohio
Tennessee
Maryland
Oregon
Connecticut
Minnesota
Wisconsin
South Carolina
Missouri
Indiana
Wyoming
Alabama
New Hampshire
Maine
Rhode Island
Alaska
New Mexico
Kentucky
Oklahoma


In [230]:
kwargs = {
    'region': Region.objects.get(name='Taiwan'),
    'year': 2017,
    'visitors': 10739000,    
}
Travel.objects.create(**kwargs)

<Travel: Travel object (298)>

# GDP

In [19]:
from django.db.models import Q
import unidecode
import pandas as pd

from casestudy.models import GDP

In [32]:
GDP.objects.all().delete()

(337, {'casestudy.GDP': 337})

In [78]:
gdpfile = '/Users/spindicate/Documents/docs/covid19/gdp/oecd-gdp-ppp-current.csv'
df_gdp = pd.read_csv(gdpfile)
header_index = df_gdp.iloc[:,0][df_gdp.iloc[:,0] == 'Year'].index[0]
df_gdp.columns = df_gdp.iloc[header_index, :]
df_gdp = df_gdp.iloc[header_index + 2: ]
regions = df_gdp['Year']
df_gdp = df_gdp.loc[:, '2016':].iloc[:,1:2]
df_gdp['regions'] = regions
df_gdp.columns = ['gdp_2016', 'regions']
df_gdp = df_gdp[['regions', 'gdp_2016']]
df_gdp['gdp_2016'] = df_gdp['gdp_2016']
tail_index = df_gdp[df_gdp.regions.str.contains('Data extracted')].index[0]
df_gdp = df_gdp.loc[:tail_index - 1]
df_gdp = df_gdp.reset_index().drop(columns='index')
df_gdp.regions = df_gdp.regions.str.split(': ', expand=True)[1]
df_gdp = df_gdp[~(df_gdp.gdp_2016 == '..')]
df_gdp.gdp_2016 = df_gdp.gdp_2016.astype('float64')

In [83]:
Region.objects.filter(name='Japan').values_list('population', flat=True)[0]

128057351

In [84]:
df_gdp[df_gdp.regions == 'Japan'].iloc[0, 1] / Region.objects.filter(name='Japan').values_list('population', flat=True)[0]

0.0406994987737955

In [86]:
df_gdp.sort_values(by='gdp_2016', ascending=False)

,regions,gdp_2016
471,China,22543900.00
383,United States,18602100.00
172,Japan,5211870.00
102,Germany,4163900.00
436,Brazil,3160800.00
...,...,...
19,Austria - not regionalised,168.84
333,Sweden - not regionalised,111.43
193,Latvia - not regionalised,71.50
81,Finland - not regionalised,65.27


In [34]:
reg_objs = []
regs_in_data = []
for i, row in df_gdp.iterrows():
    region = unidecode.unidecode(row.regions)
    if row.regions == 'Province of Trento':
        region = 'Trento'
    if row.regions == 'Province of Bolzano-Bozen':
        region = 'Bolzano/Bozen'
    if row.regions == 'Aosta Valley':
         region = "Valle d'Aosta"
            
    if region in ['Australia', 'United States', 'Italy', 'Canada', 'Brazil', 'China']:
        pass
    elif row.gdp_2016 == '..':
        pass
    else:
        try:
            if row.regions == 'Georgia' and row['gdp_2016'] == '539525':
                reg_obj = Region.objects.filter(country_key__alpha3='USA').get(Q(name=region) | Q(name_alt=region))
            else:
                reg_obj = Region.objects.get(Q(name=region) | Q(name_alt=region))
            regs_in_data.append(region)
            reg_objs.append(GDP(region=reg_obj, year=2016, gdp=float(row.gdp_2016)*1000000))
        except Region.DoesNotExist:
            pass

In [35]:
missing_names = [region.name for region in Region.objects.all() if region.name not in regs_in_data and region.name_alt not in regs_in_data]
missing = [region.id for region in Region.objects.all() if region.name not in regs_in_data and region.name_alt not in regs_in_data]

In [36]:
wbfile = '/Users/spindicate/Downloads/wb-gdp.csv'
df_wb = pd.read_csv(wbfile, skiprows=1)
df_wb = df_wb.reset_index()
df_wb.columns = df_wb.iloc[0]
df_wb = df_wb.iloc[1:]
df_wb = df_wb[['Country Name', 'Country Code', 2016.0]]
df_wb.columns = ['Country Name', 'Country Code', 'gdp_2016']

In [37]:
regs_in_data = []
for i, row in df_wb.iterrows():
    region = unidecode.unidecode(row['Country Name'])
    
    if region == 'Bahamas, The':
        region = 'Bahamas'
    if region == 'Congo, Dem. Rep.':
        region = 'Congo (Kinshasa)'
    if region == 'Congo, Rep.':
        region = 'Congo (Brazzaville)'
    if region == 'Georgia':
        region = 'Georgia, Country'
    if region == 'Macao SAR, China':
        region = 'Macau'
    if region == 'Myanmar':
        region = 'Burma'
        
    if region in ['Australia', 'United States', 'Italy', 'Canada', 'Brazil', 'China']:
        pass
    elif region in not_regions or region not in missing_names:
        pass
    else:
        try:
            regions = Region.objects.filter(id__in=missing)
            if region == 'Georgia':
                reg_obj = regions.exclude(country_key__alpha3='USA').get(Q(name=region) | Q(name_alt=region))
            else:
                reg_obj = regions.get(Q(name=region) | Q(name_alt=region))
            regs_in_data.append(region)
            reg_objs.append(GDP(region=reg_obj, year=2016, gdp=float(row.gdp_2016)))
        except Region.DoesNotExist:
            if region not in list(Region.objects.values_list('name', flat=True)):
                print (region)

In [38]:
len(reg_objs)

337

In [39]:
max_bulk_create(reg_objs)

In [15]:
not_regions = [
    'Palau',
    'Pre-demographic dividend',
    "Korea, Dem. People's Rep.",
    'Pacific island small states',
    'Post-demographic dividend',
    'South Asia',
    'Solomon Islands',
    'Sub-Saharan Africa (excluding high income)',
    'Sub-Saharan Africa',
    'Small states',
    'Sint Maarten (Dutch part)',
    'East Asia & Pacific (IDA & IBRD countries)',
    'Europe & Central Asia (IDA & IBRD countries)',
    'Tajikistan',
    'Turkmenistan',
    'Latin America & the Caribbean (IDA & IBRD countries)',
    'Middle East & North Africa (IDA & IBRD countries)',
    'Tonga',
    'South Asia (IDA & IBRD)',
    'Sub-Saharan Africa (IDA & IBRD countries)',
    'Tuvalu',
    'Upper middle income',
    'Vanuatu',
    'World',
    'Samoa',
    'Kiribati',
    'St. Kitts and Nevis',
    'Latin America & Caribbean (excluding high income)',
    'Latin America & Caribbean',
    'Least developed countries: UN classification',
    'Low income',
    'Lower middle income',
    'Low & middle income',
    'Lesotho',
    'Late-demographic dividend',
    'St. Martin (French part)',
    'Middle East & North Africa',
    'Marshall Islands',
    'Middle income',
    'Middle East & North Africa (excluding high income)',
    'North America',
    'Nauru',
    'OECD members',
    'Other small states',
    'Arab World',
    'American Samoa',
    'Central Europe and the Baltics',
    'Comoros',
    'Caribbean small states',
    'East Asia & Pacific (excluding high income)',
    'Early-demographic dividend',
    'East Asia & Pacific',
    'Europe & Central Asia (excluding high income)',
    'Europe & Central Asia',
    'Euro area',
    'European Union',
    'Fragile and conflict affected situations',
    'Micronesia, Fed. Sts.',
    'High income',
    'Heavily indebted poor countries (HIPC)',
    'IBRD only',
    'IDA & IBRD total',
    'IDA total',
    'IDA blend',
    'IDA only',
    'Not classified',
]